In [1]:
from hw1 import Composer
from midi2seq import process_midi_seq, seq2piano, random_piano, piano2seq, segment
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [35]:
sequence = process_midi_seq(maxlen=50,n=50000)
print(sequence.shape)

notes = np.unique(sequence)
print(f'number of unique notes are {len(notes)} notes')
print(notes)

(52111, 51)
number of unique notes are 302 notes
[ 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 149 150
 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186
 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204
 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222
 223 224 225 226 227 228 229 230 231 232 233 234 235 236 256 257 258 259
 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277
 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295
 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313
 3

In [39]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(-1,1))
# sequence = scaler.fit_transform(sequence)

In [42]:
X_train = sequence[:,:-1]
X_train = X_train.reshape((-1,X_train.shape[1],1))

Y_train = sequence[:,-1]
Y_train = Y_train.reshape((-1,1))

X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

X_train.shape, Y_train.shape

(torch.Size([52111, 50, 1]), torch.Size([52111, 1]))

In [75]:
torch.tensor(21 == notes).float()

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [107]:
from torch.utils.data import Dataset
class MidiComposerDataset(Dataset):
    def __init__(self,labels, x_sequence, y_next):
        self.x_sequence = x_sequence
        self.y_next = y_next
        self.labels = labels

    def __len__(self):
        return len(self.y_next)

    def one_hot_encode(self, labels, note):
        return torch.tensor(note == labels).float()
        
    def __getitem__(self, idx):
        action = int(self.y_next[idx][0])
        encode_action = self.one_hot_encode(self.labels, action)
        return dict(
            sequence = self.x_sequence[idx],
            action = encode_action
        )

In [108]:
train_dataset = MidiComposerDataset(notes, X_train,Y_train)

In [109]:
from torch.utils.data import DataLoader

BATCH_SIZE = 100

train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE, shuffle=True)

In [110]:
for _, batch in enumerate(train_loader):
    sequence_batch , action_batch = batch['sequence'].to(device) , batch['action'].to(device) 
    print(sequence_batch.shape, action_batch.shape)
    break

torch.Size([100, 50, 1]) torch.Size([100, 302])


In [111]:
class ComposerModel(nn.Module):
    def __init__(self, n_features, n_classes, n_hidden=256, n_layers=3):
        super().__init__()
        self.num_stacked_layers = n_layers
        self.hidden_size = n_hidden
        self.lstm = nn.LSTM(
            input_size = n_features,
            hidden_size = n_hidden,
            num_layers = n_layers,
            batch_first = True,
            dropout = 0.75
        )
        self.fc = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [112]:
model = ComposerModel(1, len(notes), 128, 3)
model.to(device)

ComposerModel(
  (lstm): LSTM(1, 128, num_layers=3, batch_first=True, dropout=0.75)
  (fc): Linear(in_features=128, out_features=302, bias=True)
)

In [113]:
learning_rate = 0.0001
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [117]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0
    
    for batch_index, batch in enumerate(train_loader):
        sequence_batch , action_batch = batch['sequence'].to(device) , batch['action'].to(device)
        
        output = model(sequence_batch)
        loss = loss_function(output, action_batch)
        running_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

In [ ]:
num_epochs = 200
for epoch in range(num_epochs):
    train_one_epoch()

Epoch: 1
Batch 100, Loss: 4.359
Batch 200, Loss: 4.337
Batch 300, Loss: 4.346
Batch 400, Loss: 4.349
Batch 500, Loss: 4.362

Epoch: 2
Batch 100, Loss: 4.359
Batch 200, Loss: 4.356
Batch 300, Loss: 4.358
Batch 400, Loss: 4.318
Batch 500, Loss: 4.347

Epoch: 3
Batch 100, Loss: 4.338
Batch 200, Loss: 4.346
Batch 300, Loss: 4.332
Batch 400, Loss: 4.330
Batch 500, Loss: 4.351

Epoch: 4
Batch 100, Loss: 4.309
Batch 200, Loss: 4.310
Batch 300, Loss: 4.268
Batch 400, Loss: 4.197
Batch 500, Loss: 4.197

Epoch: 5
Batch 100, Loss: 4.149
Batch 200, Loss: 4.111
Batch 300, Loss: 4.106
Batch 400, Loss: 4.090
Batch 500, Loss: 4.059

Epoch: 6
Batch 100, Loss: 3.996
Batch 200, Loss: 4.009
Batch 300, Loss: 3.981
Batch 400, Loss: 3.991
Batch 500, Loss: 3.952

Epoch: 7
Batch 100, Loss: 3.941
Batch 200, Loss: 3.905
Batch 300, Loss: 3.940
Batch 400, Loss: 3.848
Batch 500, Loss: 3.865

Epoch: 8
Batch 100, Loss: 3.846
Batch 200, Loss: 3.837
Batch 300, Loss: 3.842
Batch 400, Loss: 3.819
Batch 500, Loss: 3.851

